In [1]:
#import smodels modules

import numpy as np

from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor, TheoryPredictionsCombiner
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
import time

setLogLevel("info")

[SModelS:pyhfInterface] WARNING could not set pytorch as the pyhf backend, falling back to the default.
[SModelS:pyhfInterface] We however recommend that pytorch be installed.


In [2]:
#import taco modules
import sys, os
sys.path.insert(0, os.path.expanduser("~/taco_code"))

import numpy as np
from codes.Full_SR_Ranking.pathfinder.path_finder import PathFinder

## Input array of True/False manually

In [100]:
listOfAna = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06', 'ATLAS-SUSY-2018-32',
             'ATLAS-SUSY-2018-41','ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08', 'ATLAS-SUSY-2019-09',
             'CMS-SUS-20-004', 'CMS-SUS-21-002']
#comb_matrix = np.array([[False,True,True,True,True,True,True,True,True],
#                        [True,False,True,True,True,True,False,True,True],
#                        [True,True,False,True,False,True,True,True,True], 
#                        [True,True,True,False,True,True,True,True,True],
#                        [True,True,False,True,False,True,True,True,True],
#                        [True,True,True,True,True,False,True,True,True],
#                        [True,False,True,True,True,True,False,True,True],
#                        [True,True,True,True,True,True,True,False,False],
#                        [True,True,True,True,True,True,True,False,False]])

#np.array(~comb_matrix, dtype=int)

# Dictionary of Combinations

In [101]:
allo = {"ATLAS-SUSY-2018-05-ewk":['ATLAS-SUSY-2018-06', 'ATLAS-SUSY-2018-32','ATLAS-SUSY-2018-41',
                                  'ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08', 'ATLAS-SUSY-2019-09',
                                  'CMS-SUS-20-004', 'CMS-SUS-21-002']}
allo["ATLAS-SUSY-2018-06"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-32','ATLAS-SUSY-2018-41',
                              'ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08','CMS-SUS-20-004', 'CMS-SUS-21-002']
allo["ATLAS-SUSY-2018-32"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-41',
                              'ATLAS-SUSY-2019-08','ATLAS-SUSY-2019-09','CMS-SUS-20-004', 'CMS-SUS-21-002']
allo["ATLAS-SUSY-2018-41"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-32',
                              'ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08','ATLAS-SUSY-2019-09','CMS-SUS-20-004', 
                              'CMS-SUS-21-002']
allo["ATLAS-SUSY-2019-02"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-41',
                              'ATLAS-SUSY-2019-08','ATLAS-SUSY-2019-09','CMS-SUS-20-004', 'CMS-SUS-21-002']
allo["ATLAS-SUSY-2019-08"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-32',
                              'ATLAS-SUSY-2018-41','ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-09','CMS-SUS-20-004', 
                              'CMS-SUS-21-002']
allo["ATLAS-SUSY-2019-09"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-32','ATLAS-SUSY-2018-41',
                              'ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08','CMS-SUS-20-004', 'CMS-SUS-21-002']
allo["CMS-SUS-20-004"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-32',
                          'ATLAS-SUSY-2018-41','ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08','ATLAS-SUSY-2019-09']
allo["CMS-SUS-21-002"] = ['ATLAS-SUSY-2018-05-ewk', 'ATLAS-SUSY-2018-06','ATLAS-SUSY-2018-32',
                          'ATLAS-SUSY-2018-41','ATLAS-SUSY-2019-02','ATLAS-SUSY-2019-08','ATLAS-SUSY-2019-09'] 

## Model and Theory Predictions

In [104]:
slhafile = '/Users/sahananarasimha/smodels/ew_7q306a4m.slha'
model = Model(BSMparticles = BSMList, SMparticles = SMList)
model.updateParticles(inputFile = slhafile)

sigmacut = 0.005*fb
mingap = 5.*GeV

database = Database('/Users/sahananarasimha/smodels-database/13TeV/')

toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)

expresults = database.getExpResults(analysisIDs=listOfAna, dataTypes=['efficiencyMap','combined'])


allThPredictions = [theoryPredictionsFor(exp, toplist, combinedResults=True) for exp in expresults]


for tp in allThPredictions:
    print(tp)


INFO in model.updateParticles() in 393: Loaded 62 BSM particles
INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /Users/sahananarasimha/smodels-database/13TeV/db31.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /Users/sahananarasimha/smodels-database/13TeV/db31.pcl in 1.0 secs.


1 predictions: ATLAS-SUSY-2018-05-ewk:2.44E-06 [pb]
1 predictions: ATLAS-SUSY-2018-06:1.30E-06 [pb]
None
None
None
1 predictions: ATLAS-SUSY-2019-08:3.33E-05 [pb]
1 predictions: ATLAS-SUSY-2019-09:9.18E-06 [pb]
None
1 predictions: CMS-SUS-21-002:1.06E-04 [pb]


### Though the paper mentions that the weights are log(L_bsm/L_sm), the weights do not get registered if they are negative i..e if L_bsm < L_sm. Hence, I have taken weights to be nll ratio. Should clarify it with Jamie

In [105]:
def bestCombinationFinder(comb_matrix, theoryPrediction, expected=True):
    
    #comb_matrix = dictionary of allowed analyses combination
    #theoryPrediction = list of theory prediction objects for each expResult
    
    #print ( "type", type(theoryPrediction) )
    weight_vector = []
    
    exclMatrix = createExclusivityMatrix(comb_matrix)
    
    for tp in theoryPrediction:
        #print ( "type", type(tp) )
        if not tp:
            weight_vector.append(None)
            continue
        #get expected llhd
        for preds in tp:
            #print ( "type", type(preds) )
            lbsm = preds.likelihood(expected=True)
            lsm = preds.lsm(expected=True)
            weight = -np.log(lbsm/lsm)  #returning nll ratio
            weight_vector.append(weight)
        
        #else:
        #get observed llhd
            #lbsm = preds.likelihood()
            #lsm = preds.lsm()
            #weight = -np.log(lbsm/lsm)  #returning nll ratio
            #weight_vector.append(weight)
    
    #Get index of analyses for which theoryPrediction is None
    indices = []
    for i in range(len(weight_vector)):
        if not weight_vector[i]:
            indices.append(i)
    
    #Remove analysis from comb_matrix and weight_vector for which theoryPrediction is None
    weight_vector = np.array(weight_vector)
    weight_vector = np.delete(weight_vector, indices)
    
    exclMatrix = np.delete(np.delete(exclMatrix, indices,0), indices,1)
    
    no_tp = [theoryPrediction[i] for i in indices]
    for tp in no_tp: theoryPrediction.remove(tp) 
    
    #Get the allowed list of combinations with decreasing weights
    pf = PathFinder(np.array(~exclMatrix, dtype=int), weights=weight_vector, ignore_subset=True)
    top_paths = pf.find_path(top=5)
    
    #return list of theory predictions for which the combination has max weight
    bestCombination = [allThPredictions[i] for i in pathfin[0]['path']]
    
    return bestCombination
    
bestThPredictions = bestCombinationFinder(allo, allThPredictions)    

for tp in bestThPredictions:
    for preds in tp:
        expID = preds.expResult.globalInfo.id
        dataType = preds.dataType()
        #dataId = theoryPred.dataId()
        print ( "\nExperimental Result: %s (%s-type)" %(expID,dataType) )


Experimental Result: ATLAS-SUSY-2018-05-ewk (combined-type)

Experimental Result: ATLAS-SUSY-2019-08 (combined-type)

Experimental Result: ATLAS-SUSY-2019-09 (combined-type)

Experimental Result: CMS-SUS-21-002 (combined-type)


# True/False matrix from  Combination dictionary

In [103]:
def createExclusivityMatrix(comb_matrix):
    
    #creat a n by n True matrix where n = number of analyses in the dict
    exclMatrix = [[True for i in range(len(comb_matrix))] for i in range(len(comb_matrix))]
    
    listOfAna = [ana for ana in comb_matrix.keys()]
    
    for ana in comb_matrix.keys():
        for notcombAna in listOfAna:
            if notcombAna not in comb_matrix.get(ana):
                #print("\n ana",ana, "  notcombAna ", notcombAna)
                exclMatrix[listOfAna.index(ana)][listOfAna.index(notcombAna)] = False
                
    return np.array(exclMatrix) 
    

#allo = {"2019-09":["2019-08","2019-02"], "2019-08":["2019-09","2019-02","2018-06"], 
#       "2019-02":["2019-09","2019-08","2018-06"], "2018-06":["2019-08","2019-02"]}

#eM = createExclusivityMatrix(allo)
#eM